이 노트북은 https://bright-ocean.tistory.com/79 블로그의 코드를 조금더 stable하게 수정한 것입니다.   
최초 작성일 : 2022년 4월 14일, 임낙준 

인베스팅 닷컴의 크롤링방법을 설명하기 위해서, 먼저 해당 문서는 각 부분별로 설명을 하고

# 1. 필요한 모듈 가져오기 

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 2. 국가별 주식 종목 긁어오기

이번 2022년 헤브론스타가 인베스팅닷컴에서 갖고와야할 국가들은 아래와 같습니다. 
* 말레이시아  
* 싱가포르  
* 콜롬비아  
* 멕시코  
* 스페인  
* 이탈리아  
한 국가를 기준으로 잡고 코드를 작성하면, 코드를 국가별로 재활용 할 수 있기 때문에, 먼저 멕시코를 기준으로 코드를 작성하겠습니다.  
또한 인베스팅 닷컴은 한국어페이지도 존재하지만, 인베스팅 닷컴에서 영어표기를 우선한다고 명시해 놓았기에, 영어권 국가 사이트(예시에서는 호주)로 접속해서 멕시코의 상장사 재무정보에 접근합니다. 

### 2-1. 주식시장 페이지 접속 

아래 코드의 first_url 링크 맨 뒤에있는 'mexico'를 바꾸면 다른 국가에서도 재활용 할 수 있음

In [2]:
# 멕시코 주식시장 정보 페이지 접속
driver = webdriver.Chrome('./chromedriver.exe')
first_url = 'https://au.investing.com/equities/mexico'    #맨 뒤에 'mexico'를 바꾸면 다른 국가에서도 재활용 할 수 있음
driver.get(first_url)

### 2-2. 주식시장 페이지에서 모든 주식 종목소개 페이지로 전환

멕시코 주식 시장 페이지에 접근한 후, 페이지에 표시되는 주식을 'all stocks' 즉 모든 상장 주식으로 바꿉니다.    
이 때, 인베스팅 닷컴의 팝업 광고창 때문에 오류가 발생하는 경우가 있으니 try, except로 오류를 회피합니다.    
except에서는 광고 팝업창의 x버튼을 누르는 코드가 추가되었습니다. 

In [3]:
#페이지에 표시되는 주식을 'all stocks'로 바꿈 
try : 
    select_box = driver.find_element_by_xpath('//*[@id="stocksFilter"]').click()
    select_all_stock = driver.find_element_by_xpath('//*[@id="all"]').click()
    #time.sleep(8)

# Avoid error. 만약 광고 팝업창으로 인해 에러가 발생할 경우 except로 에러 회피
except : 
    # 광고 팝업창 닫기 
    driver.find_element_by_xpath('//*[@id="PromoteSignUpPopUp"]/div[2]/i').click()
    # try와 동일
    select_box = driver.find_element_by_xpath('//*[@id="stocksFilter"]').click()
    select_all_stock = driver.find_element_by_xpath('//*[@id="all"]').click()
    #time.sleep(8)

### 2-3. 모든 주식 종목 소개 페이지에서 각 종목(기업)별 상세 정보 페이지 링크 추출

인베스팅 닷컴은 동적 페이지입니다. 모든 주식 종목 코드를 긁어 올 수 있도록, 스크롤을 끝까지 내렸다가 다시 올립니다. 

In [4]:
# 인베스팅 닷컴은 동적페이지이다. 
# BeautifulSoup을 사용할수 있도록 스크롤을 한번 끝까지 내렸다 올린다.
SCROLL_PAUSE_SEC = 3

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

주식 종목 표를 Beautifulsoup을 이용해 갖고 옵니다. 이후 상세 정보 페이지의 링크를 추출하여, company_links란 리스트에 저장합니다.

In [5]:
soup = BeautifulSoup(driver.page_source,"html.parser")

# 주식 테이블을 가지고 온다. 
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
co
# 메인 주식 테이블에서 회사 페이지로
all_atag_maintable = maintable.find_all('a')

# soup의 find 메서드를 이용해서 갖고 오게 되면, 링크 뿐 아니라 다른 정보들도 같이 갖고 오게 됩니다. 
# 따라서 아래 for문으로 순수한 href 링크만을 추출하여, company_links에 저장합니다. 
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"] #find로 한번더 돌려준다.
    company_links.append(company_link)

In [6]:
len(company_links)

35

# 3. 기업별 재무정보 추출

### 3-1. 재무정보 페이지 링크  생성  
앞서 company_links에 기업별 상세 정보 페이지 링크를 추출하였습니다. 기업별 상세 정보 페이지 링크에 쿼리스트링처럼, 적절한 인자를 연결하면 재무정보에 접근할 수 있습니다.  

ex) America Movil이라는 멕시코 기업 :  
base url 뒤에 America Movil의 상세페이지로 넘어가는 company link를 붙이고, 손익계산서를 의미하는 -income-statement를 추가하게 되면 America Movil의 손익계산서 페이지로 넘어가게 됩니다.  

'https://au.investing.com/'  (baseurl) + 'equities/america-movil-l' (company link : Mmerica movil 상세정보페이지) + '-income-statement'   
=  https://au.investing.com/equities/america-movil-l-income-statement

손익계산서, 재무상태표(대차대조표), 현금흐름표에 해당하는 접미사들은 아래 코드와 같습니다.   
base_url + company_link + 원하는 표의 suffix   로 http 링크를 만들어 이동합니다. 

In [7]:
base_url = 'https://au.investing.com'

IS_suffix = '-income-statement'
BS_suffix = '-balance-sheet'
CF_suffix = '-cash-flow'

### 3-2. 추출하고자 하는 계정과목(칼럼) 설정
인베스팅 닷컴에서 확인할 수 있는 계정과목들은 많습니다. 2022년 BDC과제에서 작성한 테이블 명세서에 근거하여 추출하고자 하는 계정과목이름들을 리스트에 저장합니다. 해당 리스트들은 3-3에서 사용됩니다.

In [8]:
# 추출하고자 하는 칼럼들 

is_targets =['Total Revenue','Cost of Revenue, Total','Selling/General/Admin. Expenses, Total','Operating Income',
             'Interest Income (Expense), Net Non-Operating', 'Interest Expense (Income) - Net Operating',
            'Net Income Before Taxes','Net Income After Taxes']

bs_targets = ['Total Current Assets','Total Assets','Total Current Liabilities','Total Liabilities','Total Equity']

cf_targets = ['Cash From Operating Activities','Cash From Financing Activities','Cash From Investing Activities']


### 3-3. 정보 추출
크롤링의 핵심이 되는 내용입니다. 손익계산서, 대차대조표, 현금흐름표의 데이터를 추출하고 전처리 하는 과정이 모두 같기 때문에 손익계산서 추출 코드를 설명한 후, 마지막 part4에서 이를 다른 재무제표에 대해서도 적용해보겠습니다. 

In [9]:
# 예시를 위해 company_links에서 기업 하나를 갖고 옴 
company = company_links[0]

3-1에서 설명한대로 base_url+company+IS_suffix 로 손익계산서 페이지에 접근하는 링크를 만들고, 브라우저를 손익계산서 페이지로 이동시킵니다.

In [10]:
#income statement 
driver.get(base_url+company+IS_suffix)
time.sleep(1)

연간 재무제표를 추출해 올 것이기 때문에, Annual 버튼을 클릭합니다.

In [ ]:
# 'Annual', 'Quaterly' 버튼 중 'Annual' Click
driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()

pd.read_html은 html 소스코드에서 table 속성에 해당하는 값을 가져오는 함수입니다. 
pd.read_html을 이용하여 웹페이지에 있는 표를 갖고 온 후, df_income 데이터프레임을 생성합니다. 

In [ ]:
df_income = pd.read_html(driver.page_source)[0].dropna()
df_income.head()

df_income 데이터프레임에 회사이름, 재무제표에서 사용한 통화 단위도 추가합니다. 

In [ ]:
df_income_unit = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[10]').text             
company_name = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[1]/h1').text

### 3-4. 데이터프레임 전처리 
3-3에서 추출한 데이터 프레임을 바로 사용할 수 없습니다. 우리가 원하는 형태로 데이터 프레임을 적절히 전처리 합니다. 

In [116]:
# 행렬 전치. Matrix Transpose
df_income = df_income.T
df_income.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
Period Ending:,Total Revenue,Revenue 855534.87 1016886.53 1007347.87 103820...,Revenue,"Other Revenue, Total","Cost of Revenue, Total",Gross Profit,Total Operating Expenses,"Selling/General/Admin. Expenses, Total 180838....","Selling/General/Admin. Expenses, Total",Research & Development,...,Total Extraordinary Items,Net Income,Total Adjustments to Net Income,Income Available to Common Excluding Extraordi...,Dilution Adjustment,Diluted Net Income,Diluted Weighted Average Shares,Diluted EPS Excluding Extraordinary Items,DPS - Common Stock Primary Issue,Diluted Normalized EPS
202131/12,855534.87,Revenue 855534.87 1016886.53 1007347.87 103820...,855534.87,-,503686.53,351848.34,689402.23,"Selling/General/Admin. Expenses, Total 180838....",180838.41,-,...,125312.83,196025.28,-,70712.45,-,196025.28,64689.74,1.09,0.4,1.09
202031/12,1016886.53,Revenue 855534.87 1016886.53 1007347.87 103820...,1016886.53,-,470427.48,546459.05,851531.62,"Selling/General/Admin. Expenses, Total 180838....",212135.83,-,...,-,46852.61,-,46852.61,-,46852.61,66862.56,0.7,0.38,0.7
201931/12,1007347.87,Revenue 855534.87 1016886.53 1007347.87 103820...,1007347.87,-,471736.16,535611.71,852507.33,"Selling/General/Admin. Expenses, Total 180838....",215993.86,-,...,-,67730.89,-,67730.89,-,67730.89,66004.21,1.03,0.35,1.03
201831/12,1038207.68,Revenue 855534.87 1016886.53 1007347.87 103820...,1038207.68,-,508822.43,529385.25,898650.51,"Selling/General/Admin. Expenses, Total 180838....",227192.48,-,...,-,52566.2,-,52566.2,-,52566.2,66034.79,0.8,0.32,0.8


In [117]:
# 계정과목을 column 이름으로 설정
df_income = df_income.rename(columns = df_income.iloc[0])
# 0번째 row를 삭제하고 나머지만 남긴다.
df_income = df_income.iloc[1:, :]
df_income.head()

,Total Revenue,"Revenue 855534.87 1016886.53 1007347.87 1038207.68 Other Revenue, Total - - - -",Revenue,"Other Revenue, Total","Cost of Revenue, Total",Gross Profit,Total Operating Expenses,"Selling/General/Admin. Expenses, Total 180838.41 212135.83 215993.86 227192.48 Research & Development - - - - Depreciation / Amortization - 164243.68 158915.21 155712.58 Interest Expense (Income) - Net Operating - - - - Unusual Expense (Income) - - - - Other Operating Expenses, Total 4877.29 4724.63 5862.1 6923.02","Selling/General/Admin. Expenses, Total",Research & Development,...,Total Extraordinary Items,Net Income,Total Adjustments to Net Income,Income Available to Common Excluding Extraordinary Items,Dilution Adjustment,Diluted Net Income,Diluted Weighted Average Shares,Diluted EPS Excluding Extraordinary Items,DPS - Common Stock Primary Issue,Diluted Normalized EPS
202131/12,855534.87,Revenue 855534.87 1016886.53 1007347.87 103820...,855534.87,-,503686.53,351848.34,689402.23,"Selling/General/Admin. Expenses, Total 180838....",180838.41,-,...,125312.83,196025.28,-,70712.45,-,196025.28,64689.74,1.09,0.4,1.09
202031/12,1016886.53,Revenue 855534.87 1016886.53 1007347.87 103820...,1016886.53,-,470427.48,546459.05,851531.62,"Selling/General/Admin. Expenses, Total 180838....",212135.83,-,...,-,46852.61,-,46852.61,-,46852.61,66862.56,0.7,0.38,0.7
201931/12,1007347.87,Revenue 855534.87 1016886.53 1007347.87 103820...,1007347.87,-,471736.16,535611.71,852507.33,"Selling/General/Admin. Expenses, Total 180838....",215993.86,-,...,-,67730.89,-,67730.89,-,67730.89,66004.21,1.03,0.35,1.03
201831/12,1038207.68,Revenue 855534.87 1016886.53 1007347.87 103820...,1038207.68,-,508822.43,529385.25,898650.51,"Selling/General/Admin. Expenses, Total 180838....",227192.48,-,...,-,52566.2,-,52566.2,-,52566.2,66034.79,0.8,0.32,0.8


3-2에서 만든 추출하고자 하는 데이터 컬럼명들로 df_income에서 필요한 열들만 뽑아냅니다. 여기까지 오면 필요한 정보들이 다 추가된 것 같습니다.

In [120]:
df_income = df_income[is_targets]
df_income

,Total Revenue,"Cost of Revenue, Total","Selling/General/Admin. Expenses, Total",Operating Income,"Interest Income (Expense), Net Non-Operating",Interest Expense (Income) - Net Operating,Net Income Before Taxes,Net Income After Taxes
202131/12,855534.87,503686.53,180838.41,166132.64,-55877.62,-,102760.16,74615.4
202031/12,1016886.53,470427.48,212135.83,165354.91,-97961.8,-,67393.11,51026.96
201931/12,1007347.87,471736.16,215993.86,154840.54,-33493.55,-,121346.99,70313.46
201831/12,1038207.68,508822.43,227192.48,139557.17,-38563.27,-,100993.9,54516.82


마지막으로, 데이터 프레임에 회계연도, 회사명, 통화단위 칼럼을 추가합니다.

In [121]:
df_income['df_income_unit'] = df_income_unit                 # 데이터프레임에 income statement 통화단위 삽입
df_income['time']= df_income.index                           # 데이터프레임에 '시간' 추가
df_income['company_name'] = company_name                     # 데이터 프레임에 회사 이름 삽입
df_income=df_income.reset_index(drop=True)                  # index 초기화 

In [122]:
df_income

,Total Revenue,"Cost of Revenue, Total","Selling/General/Admin. Expenses, Total",Operating Income,"Interest Income (Expense), Net Non-Operating",Interest Expense (Income) - Net Operating,Net Income Before Taxes,Net Income After Taxes,df_income_unit,time,company_name
0,855534.87,503686.53,180838.41,166132.64,-55877.62,-,102760.16,74615.4,* In Millions of MXN (except for per share items),202131/12,America Movil SAB de CV (AMXL)
1,1016886.53,470427.48,212135.83,165354.91,-97961.8,-,67393.11,51026.96,* In Millions of MXN (except for per share items),202031/12,America Movil SAB de CV (AMXL)
2,1007347.87,471736.16,215993.86,154840.54,-33493.55,-,121346.99,70313.46,* In Millions of MXN (except for per share items),201931/12,America Movil SAB de CV (AMXL)
3,1038207.68,508822.43,227192.48,139557.17,-38563.27,-,100993.9,54516.82,* In Millions of MXN (except for per share items),201831/12,America Movil SAB de CV (AMXL)


# 4. 전과정 자동화, 최종 완성코드

Part 1,2,3에서 설명했던 것을 for문을 이용해서 자동화 합니다. 


In [11]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

'first_url' 맨 뒤의 'mexico'를 바꾸면 다른 국가에서도 재활용 할 수 있음

In [12]:
# 멕시코 주식시장 정보 페이지 접속
driver = webdriver.Chrome('./chromedriver.exe')
first_url = 'https://au.investing.com/equities/mexico'    #맨 뒤에 'mexico'를 바꾸면 다른 국가에서도 재활용 할 수 있음
driver.get(first_url)

In [13]:
#페이지에 표시되는 주식을 'all stocks'로 바꿈 
try : 
    select_box = driver.find_element_by_xpath('//*[@id="stocksFilter"]').click()
    select_all_stock = driver.find_element_by_xpath('//*[@id="all"]').click()
    #time.sleep(8)

# Avoid error. 만약 광고 팝업창으로 인해 에러가 발생할 경우 except로 에러 회피
except : 
    # 광고 팝업창 닫기 
    driver.find_element_by_xpath('//*[@id="PromoteSignUpPopUp"]/div[2]/i').click()
    # try와 동일
    select_box = driver.find_element_by_xpath('//*[@id="stocksFilter"]').click()
    select_all_stock = driver.find_element_by_xpath('//*[@id="all"]').click()
    #time.sleep(8)

In [14]:
# 인베스팅 닷컴은 동적페이지이다. 
# BeautifulSoup을 사용할수 있도록 스크롤을 한번 끝까지 내렸다 올린다.
SCROLL_PAUSE_SEC = 3

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [15]:
soup = BeautifulSoup(driver.page_source,"html.parser")

# 주식 테이블을 가지고 온다. 
maintable = soup.find('table', id='cross_rate_markets_stocks_1')

# 메인 주식 테이블에서 회사 페이지로
all_atag_maintable = maintable.find_all('a')

# soup의 find 메서드를 이용해서 갖고 오게 되면, 링크 뿐 아니라 다른 정보들도 같이 갖고 오게 됩니다. 
# 따라서 아래 for문으로 순수한 href 링크만을 추출하여, company_links에 저장합니다. 
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"] #find로 한번더 돌려준다.
    company_links.append(company_link)

In [16]:
base_url = 'https://au.investing.com'

IS_suffix = '-income-statement'
BS_suffix = '-balance-sheet'
CF_suffix = '-cash-flow'

In [17]:
# 추출하고자 하는 칼럼들 

is_targets =['Total Revenue','Cost of Revenue, Total','Selling/General/Admin. Expenses, Total','Operating Income',
             'Interest Income (Expense), Net Non-Operating', 'Interest Expense (Income) - Net Operating',
            'Net Income Before Taxes','Net Income After Taxes']

bs_targets = ['Total Current Assets','Total Assets','Total Current Liabilities','Total Liabilities','Total Equity']

cf_targets = ['Cash From Operating Activities','Cash From Financing Activities','Cash From Investing Activities']


In [18]:
#cross_rate_markets_stocks_1

result_df = pd.DataFrame()
crawling_failed_companies = []
for company in company_links : 
    
    try : 
        #income data
        driver.get('https://au.investing.com'+company+IS_suffix)
        time.sleep(2)
        driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        df_income = pd.read_html(driver.page_source)[0].dropna()
        df_income_unit = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[10]').text             # 통화 단위(Unit)
        company_name = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[1]/h1').text             # 회사이름 추출

        # income statement dataframe preprocess
        df_income = df_income.T
        df_income = df_income.rename(columns = df_income.iloc[0])         # 계정과목을 column 이름으로 설정
        df_income = df_income.iloc[1:, :]                                 # 0번째 row를 삭제하고 나머지만 남긴다.
        df_income = df_income[is_targets]
        df_income['df_income_unit'] = df_income_unit                 # 데이터프레임에 income statement 통화단위 삽입
        df_income['df_income_time']= df_income.index                           # 데이터프레임에 '시간' 추가
        df_income['company_name'] = company_name                     # 데이터 프레임에 회사 이름 삽입
        df_income=df_income.reset_index(drop=True)


        # balance data
        driver.get('https://au.investing.com'+company+BS_suffix)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="leftColumn"]/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        df_balance = pd.read_html(driver.page_source)[0].dropna()
        df_balance_unit = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[10]').text           # 통화 단위(Unit) 추출

        # balance sheet dataframe preprocess
        df_balance = df_balance.T
        df_balance = df_balance.rename(columns = df_balance.iloc[0])      # 계정과목을 column 이름으로 설정
        df_balance = df_balance.iloc[1:, :]                               # 0번째 row를 삭제하고 나머지만 남긴다.
        df_balance = df_balance[bs_targets]
        df_balance['balance_unit'] = df_balance_unit                 # 데이터프레임에 cash flow 통화단위 삽입
        df_balance['df_balance_time']= df_balance.index
        df_balance=df_balance.reset_index(drop=True)

        #cash flow
        driver.get('https://au.investing.com'+company+CF_suffix)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="leftColumn"]/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        df_cash_flow = pd.read_html(driver.page_source)[0].dropna()
        df_cash_flow_unit = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[10]').text            # 통화 단위(Unit) 추출

        #cash flow dataframe preprocess
        df_cash_flow = df_cash_flow.T 
        df_cash_flow = df_cash_flow.rename(columns = df_cash_flow.iloc[0]) # 계정과목을 column 이름으로 설정
        df_cash_flow = df_cash_flow.iloc[1:, :]                            # 0번째 row를 삭제하고 나머지만 남긴다.
        df_cash_flow = df_cash_flow[cf_targets]
        df_cash_flow['cash_flow_unit'] = df_cash_flow_unit                 # 데이터프레임에 cash flow 통화단위 삽입
        df_cash_flow['df_cash_flow']= df_cash_flow.index
        df_cash_flow = df_cash_flow.reset_index(drop=True)

        company_df = pd.concat([df_balance, df_income, df_cash_flow],axis=1)

        # Append
        result_df = result_df.append(company_df, ignore_index = True)
        
    except : 
        crawling_failed_companies.append(company)
        pass
        
    
    

In [ ]:
result_df.to_excel('./investingDotcom Crawling Colombia.xlsx')

In [142]:
# 크롤링에 성공한 기업 수 
552/4

138.0

In [139]:
# 멕시코 상장사 기업 수 
len(company_links)

671

In [141]:
# 크롤링에 실패한 기업들 
len(crawling_failed_companies)

533

### Question) 저번에 조사한 멕시코의 상장기업수는 140개

해결해야할 문제 :   
* 은행은 일반기업들과 다른 계정과목을 사용함. 위 for 반복문이 은행의 경우 에러가 떠버린다. 
- try except를 이용하여 회피한다. 설명하는 부분에도 이를 추가해야한다. 
* 은행의 계정과목을 테이블 명세서에 반영해야겠다. 
* 스크롤이 과연 끝까지 되는지 확인해야한다. 호주의 경우 상장사가 1000개가 넘는데, 크롤링되는 주식종목은 정확히 1000이다. 
* 테이블을 가공해서 만들어야할 
* time.sleep(waittime)의 waittime을 함수의 변수로 빼버려서 기다리는 시간을 조절하기 쉽게 만들자. 
